In [1]:
# Setup
import sys
from pathlib import Path

# Agregar directorio raíz al path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print(f"✅ Project root: {project_root}")

✅ Project root: D:\Academico\SENATI\octavo_ciclo\appBigData


In [2]:
# Imports
from sqlalchemy import create_engine, text
from config.config import Config
from src.logger_config import get_logger
import pandas as pd

logger = get_logger('notebook_extraccion')
logger.info("🚀 Iniciando proceso de extracción")

✅ Configuración cargada correctamente
04:18:24 | notebook_extraccion | INFO | 🚀 Iniciando proceso de extracción


In [3]:
# Celda 3: Crear schema de staging
from sqlalchemy import text

engine_staging = create_engine(Config.get_staging_connection_string())

logger.info("🔨 Creando schema de staging...")

# Crear tabla etl_control
sql_etl_control = """
CREATE TABLE IF NOT EXISTS etl_control (
    etl_id INT AUTO_INCREMENT PRIMARY KEY,
    proceso VARCHAR(100) NOT NULL,
    fecha_inicio DATETIME NOT NULL,
    fecha_fin DATETIME,
    estado ENUM('INICIADO', 'COMPLETADO', 'ERROR') DEFAULT 'INICIADO',
    registros_leidos INT DEFAULT 0,
    registros_escritos INT DEFAULT 0,
    registros_error INT DEFAULT 0,
    mensaje_error TEXT,
    duracion_segundos INT,
    INDEX idx_proceso (proceso),
    INDEX idx_fecha (fecha_inicio)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4
"""

# Crear tabla audit_calidad
sql_audit = """
CREATE TABLE IF NOT EXISTS audit_calidad (
    audit_id INT AUTO_INCREMENT PRIMARY KEY,
    etl_id INT,
    tabla_origen VARCHAR(100),
    tabla_destino VARCHAR(100),
    validacion VARCHAR(255),
    resultado ENUM('PASS', 'FAIL', 'WARNING'),
    valor_esperado VARCHAR(100),
    valor_obtenido VARCHAR(100),
    mensaje TEXT,
    fecha_validacion DATETIME DEFAULT CURRENT_TIMESTAMP,
    INDEX idx_etl (etl_id),
    INDEX idx_resultado (resultado)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4
"""

with engine_staging.connect() as conn:
    # Crear tablas de control
    conn.execute(text(sql_etl_control))
    conn.commit()
    logger.info("✅ Tabla etl_control creada")
    
    conn.execute(text(sql_audit))
    conn.commit()
    logger.info("✅ Tabla audit_calidad creada")

# Las tablas stg_* se crearán automáticamente cuando el extractor haga to_sql()
logger.info("✅ Schema básico de staging listo")

# Verificar
query_check = text("SHOW TABLES")
with engine_staging.connect() as conn:
    result = conn.execute(query_check)
    tablas = [row[0] for row in result]
    logger.info(f"📋 Tablas existentes: {tablas}")

04:18:25 | notebook_extraccion | INFO | 🔨 Creando schema de staging...
04:18:25 | notebook_extraccion | INFO | ✅ Tabla etl_control creada
04:18:25 | notebook_extraccion | INFO | ✅ Tabla audit_calidad creada
04:18:25 | notebook_extraccion | INFO | ✅ Schema básico de staging listo
04:18:25 | notebook_extraccion | INFO | 📋 Tablas existentes: ['audit_calidad', 'etl_control', 'stg_address', 'stg_category', 'stg_city', 'stg_country', 'stg_film', 'stg_film_category', 'stg_inventory', 'stg_payment', 'stg_rental', 'stg_store']


In [4]:
# Importar extractor
import importlib
import src.extractor

# Recargar el módulo para que tome los cambios
importlib.reload(src.extractor)

from src.extractor import SakilaExtractor

extractor = SakilaExtractor()
logger.info("✅ Extractor inicializado")

04:18:29 | extractor | INFO | ✅ Extractor inicializado correctamente
04:18:29 | notebook_extraccion | INFO | ✅ Extractor inicializado


In [5]:
# Extracción completa (primera vez)
"""
Primera extracción: COMPLETA
Extrae todas las tablas de Sakila y las carga a Staging
"""
estadisticas = extractor.extraer_todas_las_tablas(
    incremental=False,
    fecha_desde=None
)

print("\n" + "="*80)
print("📊 RESUMEN DE EXTRACCIÓN")
print("="*80)

total_registros = 0
for tabla, stats in estadisticas.items():
    if 'error' in stats:
        print(f"❌ {tabla}: ERROR - {stats['error']}")
    else:
        print(f"✅ {tabla}: {stats['leidos']:,} registros extraídos")
        total_registros += stats['leidos']

print("="*80)
print(f"📦 TOTAL: {total_registros:,} registros extraídos")
print("="*80)

04:18:32 | extractor | INFO | 📝 ETL registrado con ID: 4
04:18:32 | extractor | INFO | ================================================================================
04:18:32 | extractor | INFO | 🚀 INICIO: EXTRACCION_COMPLETA
04:18:32 | extractor | INFO |    Extrayendo 10 tablas
04:18:32 | extractor | INFO | ================================================================================
04:18:32 | extractor | INFO | 📥 Extrayendo rental (completo)
04:18:32 | extractor | INFO | ✅ Extraídos 16,044 registros de rental
04:18:36 | extractor | INFO | ✅ Cargados 16,044 registros a stg_rental
04:18:36 | extractor | INFO | 📊 Tabla: rental
04:18:36 | extractor | INFO |    Filas leídas: 16,044
04:18:36 | extractor | INFO |    Filas escritas: 16,044
04:18:36 | extractor | INFO | 📥 Extrayendo payment (completo)
04:18:36 | extractor | INFO | ✅ Extraídos 16,044 registros de payment
04:18:39 | extractor | INFO | ✅ Cargados 16,044 registros a stg_payment
04:18:39 | extractor | INFO | 📊 Tabla: payment

In [7]:
# Verificar datos en staging
from sqlalchemy import text

"""
Verificar que los datos se cargaron correctamente
"""
query_verificacion = text("""
    SELECT 
        TABLE_NAME as tabla,
        TABLE_ROWS as registros_aproximados
    FROM information_schema.TABLES
    WHERE TABLE_SCHEMA = 'sakila_staging'
    AND TABLE_NAME LIKE 'stg_%'
    ORDER BY TABLE_ROWS DESC
""")

with engine_staging.connect() as conn:
    df_verificacion = pd.read_sql(query_verificacion, conn)

print("\n📋 Tablas en Staging:")
print(df_verificacion.to_string(index=False))



📋 Tablas en Staging:
            tabla  registros_aproximados
       stg_rental                  16100
      stg_payment                  16055
    stg_inventory                   4585
         stg_film                   1000
stg_film_category                   1000
      stg_address                    603
         stg_city                    600
      stg_country                    109
     stg_category                     16
        stg_store                      2


In [8]:
# Ver muestra de datos (ejemplo: rental)
"""
Ver una muestra de los datos en staging
"""
query_muestra = """
    SELECT * FROM stg_rental 
    ORDER BY rental_date DESC 
    LIMIT 10
"""
df_muestra = pd.read_sql(query_muestra, engine_staging)
print("\n📊 Muestra de stg_rental:")
display(df_muestra)


📊 Muestra de stg_rental:


,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update,etl_fecha_carga,etl_id
0,11577,2006-02-14 15:16:03,4106,219,None,2,2006-02-15 21:30:53,2025-10-04 04:18:33,4
1,11646,2006-02-14 15:16:03,478,11,None,2,2006-02-15 21:30:53,2025-10-04 04:18:33,4
2,11541,2006-02-14 15:16:03,2026,335,None,1,2006-02-15 21:30:53,2025-10-04 04:18:33,4
3,11672,2006-02-14 15:16:03,3947,521,None,2,2006-02-15 21:30:53,2025-10-04 04:18:33,4
4,11496,2006-02-14 15:16:03,2047,155,None,1,2006-02-15 21:30:53,2025-10-04 04:18:33,4
5,11611,2006-02-14 15:16:03,1857,192,None,2,2006-02-15 21:30:53,2025-10-04 04:18:33,4
6,11593,2006-02-14 15:16:03,817,99,None,1,2006-02-15 21:30:53,2025-10-04 04:18:33,4
7,11563,2006-02-14 15:16:03,1545,83,None,1,2006-02-15 21:30:53,2025-10-04 04:18:33,4
8,11652,2006-02-14 15:16:03,1622,597,None,2,2006-02-15 21:30:53,2025-10-04 04:18:33,4
9,11657,2006-02-14 15:16:03,3043,53,None,2,2006-02-15 21:30:53,2025-10-04 04:18:33,4


In [9]:
# Ver control de ejecuciones
"""
Revisar la tabla de control ETL
"""
query_control = """
    SELECT 
        etl_id,
        proceso,
        fecha_inicio,
        fecha_fin,
        estado,
        registros_leidos,
        registros_escritos,
        duracion_segundos
    FROM etl_control
    ORDER BY etl_id DESC
    LIMIT 5
"""

df_control = pd.read_sql(query_control, engine_staging)
print("\n📋 Últimas ejecuciones ETL:")
display(df_control)


📋 Últimas ejecuciones ETL:


,etl_id,proceso,fecha_inicio,fecha_fin,estado,registros_leidos,registros_escritos,duracion_segundos
0,4,EXTRACCION_COMPLETA,2025-10-04 04:18:32,2025-10-04 04:18:47,COMPLETADO,39999,39999,15.0
1,3,EXTRACCION_COMPLETA,2025-10-04 04:15:47,2025-10-04 04:16:05,COMPLETADO,39396,39396,18.0
2,2,EXTRACCION_COMPLETA,2025-10-04 04:11:31,2025-10-04 04:12:04,COMPLETADO,39396,39396,33.0
3,1,EXTRACCION_COMPLETA,2025-10-04 04:04:57,NaT,INICIADO,0,0,NaN


In [10]:
# Prueba de extracción incremental (simulación)
"""
Simular una extracción incremental
(En producción, esto se ejecutaría mensualmente)
"""
# Obtener última fecha
ultima_fecha = extractor.obtener_ultima_extraccion()
print(f"\n📅 Última extracción: {ultima_fecha}")

# Para probar incremental, necesitarías tener datos nuevos en Sakila
# Por ahora, solo mostramos cómo se llamaría:
print("\n💡 Para extracción incremental, usarías:")
print("estadisticas_inc = extractor.extraer_todas_las_tablas(")
print("    incremental=True,")
print("    fecha_desde=ultima_fecha")
print(")")

04:20:19 | extractor | INFO | 📅 Última extracción: 2025-10-04 04:18:47

📅 Última extracción: 2025-10-04 04:18:47

💡 Para extracción incremental, usarías:
estadisticas_inc = extractor.extraer_todas_las_tablas(
    incremental=True,
    fecha_desde=ultima_fecha
)


In [11]:
# Cerrar conexiones
extractor.cerrar_conexiones()
logger.info("✅ Proceso de extracción completado")

print("\n" + "="*80)
print("🎉 ¡EXTRACCIÓN COMPLETADA EXITOSAMENTE!")
print("="*80)
print("\n📌 Siguiente paso: 02_staging.ipynb (Validaciones y limpieza)")

04:20:25 | extractor | INFO | 🔌 Conexiones cerradas
04:20:25 | notebook_extraccion | INFO | ✅ Proceso de extracción completado

🎉 ¡EXTRACCIÓN COMPLETADA EXITOSAMENTE!

📌 Siguiente paso: 02_staging.ipynb (Validaciones y limpieza)
